In [1]:
import json
from collections import Counter, defaultdict
from wiki_labels import qid_lab_get, qid_en_desc_get
from wiki_location import q2cc
from text_utils import cl
import sys
sys.path.insert(0, '/projekti/mondoAPI')
from pnu.parse import parse
from api.db import db
from tqdm.notebook import tqdm

print(qid_lab_get(42, 'en').keys(), parse('davor lauc')['tags'][0], parse('반기문')['tags'][0])

dict_keys(['douglas adams']) {'tokens': ['davor', 'lauc'], 'labels': ('fn1_hr_Latn_HR', 'ln_hr_Latn_HR'), 'counts': 1041.249228961059, 'prob': 93.48} {'tokens': ['반', '기문'], 'labels': ('ln_ko_Hang_KR', 'fn1_ko_Hang_KR'), 'counts': 2159.896804479325, 'prob': 94.46}


In [ ]:
# wiki lang to iso
WIKI_DIR = '/backup/wikidata'
api_key = open('airtable_key.txt').read()
from pyairtable import Table, Api
import sys
sys.path.insert(0, '/projekti/nelma')
from mondoDB.referencedb import provi

if True:
    lang2cc = defaultdict(Counter)
    for k, v in provi.items():
        lang, s, c = v['id'].split('_')
        bod = float(v['Percent'])*3 if 'Percent' in v else 0
        if 'Status' in v:
            if v['Status'] == 'official':
                bod += 1
            elif v['Status'] == 'official_regional':
                bod += 0.5
            elif v['Status'] == 'de_facto_official':
                bod += 0.9
            elif v['Status'] == 'romanized':
                bod += 0.2
            elif v['Status'] == 'foreign':
                bod += -0.5
        if bod>1.1:
            lang2cc[f'{lang}'][c] = max(bod, lang2cc[f'{lang}'][c])
    api = Api(api_key)
    w2iso = {t['fields']['WMF']: t['fields']['qid'] if 'qid' in t['fields'] else None for t in Table(api_key, 'appUZvAm9EHZgC1Eg', 'wiki_lang').all()}
    wiki2cc = defaultdict(Counter)
    for k, v in w2iso.items():
        if len(k) == 2:
            wiki2cc[v].update(lang2cc[k])
        elif k.count('-') == 1:
            a, b = k.split('-')
            if len(a) == 2 and len(b) == 2:
                wiki2cc[v][b.upper()] += 5 
            elif len(a) == 2 and len(b) == 4:
                #wiki2cc[v].update(lang2cc[f'{k}_{b.uppser()}'])
                wiki2cc[v].update(lang2cc[a])
    wikil2cc = {k: {cc: round(100*v2/max(v.values())) for cc, v2 in v.items()} for k, v in wiki2cc.items() if len(v)>0 and max(v.values())>0}

cc2lang = defaultdict(set)
for lang, ccs in lang2cc.items():
    for cc in ccs:
        if ccs[cc]>1:
            cc2lang[cc].add(lang)
iso2w = {k:v for v, k in w2iso.items()}

wikil2cc['Q1860']['US'], cc2lang['CH']

In [ ]:
def get_wiki_cc(country, birth_death_place, language):
    ccs = Counter()
    for bp in birth_death_place:
        if bp[5:] in q2cc:
            ccs[q2cc[bp[5:]]] += 1
    for cq in j['country']:
        if cq[5:] in q2cc:
            ccs[q2cc[cq[5:]]] += 2
    for lng in language:
        lng = lng[5:]
        if lng in wikil2cc:
            for cl,bod in wikil2cc[lng].items():
                ccs[cl] += bod*2 / 100
    cc,_ = ccs.most_common()[0] if len(ccs)>0 else ('', '')
    return cc


In [ ]:
fo = open('/backup/wikidata/wiki_person.tsv', 'w')
fo.write('qid\tname\tfn\tln\tgender\tdesc\tplace\tdob\timage\tsort\n')
for i, l in tqdm(enumerate(open('/backup/wikidata/wikinelma.jsonl')), total=17_630_406):
    j = json.loads(l)
    if j['type'] == 'per':
        qid = int(j['wiki_id'][1:])
        cc = get_wiki_cc(j['country']+j['nationality'], j['residence']+j['birth_place']+j['death_place'], j['native_language'])
        name = None
        if j['name_native']:
            name = cl(j['name_native'][0]).lower()
        else:
            if j['native_language']:
                langs = Counter([iso2w[q[5:]] for q in j['native_language'] if q[5:] in iso2w])
            else:
                langs = Counter()
            langs.update(cc2lang[cc])
            for l, f in langs.most_common():
                name = qid_lab_get(qid, lang=l)
                if name:
                    name = list(name)[0]
                    break
        if not name:
            try:
                name = list(qid_lab_get(qid, 'en', include_alt=False).keys())[0]
            except:
                try:
                    name = list(j['l'].values())[0][0]
                except:
                    name = ''
        if name:
            namel = name.lower().strip()
            desc = qid_en_desc_get(qid)
            dob = j['dob'][0][:4] if j['dob'] else ''
            image = j['picture'][0] if j['picture'] else ''
            sort = (int(dob) if dob else 1980)-(50 if image else 0)-(10 if desc else 0)

            fn = ''
            if j['fn']:
                fns = qid_lab_get(int(j['fn'][0][6:]), include_alt=True).keys()
                fns = Counter({a:len(a) for a in fns if a in namel})
                if len(fns)>0:
                    fn = fns.most_common()[0][0]

            ln = ''
            if j['ln']:
                fns = qid_lab_get(int(j['ln'][0][6:]), include_alt=True).keys()
                fns = Counter({a:len(a) for a in fns if a in namel})
                if len(fns)>0:
                    ln = fns.most_common()[0][0]
            # only 
            gender = ''
            if j['gender']==['WIKI_Q6581097']:
                gender = 'm'
            elif j['gender']==['WIKI_Q6581072']:
                gender = 'f'
            
            if False and not fn or not ln:
                if ',' in namel:
                    if ' ' in namel.split(',')[0].strip():
                        namel = namel.split(',')[0].strip()
                    else:
                        namel = namel.split(',')[1].strip()+' '+namel.split(',')[0].strip()
                    #print(name, namel)
                parts = namel.split(' ')
                if len(parts) == 1 and fn:
                    if namel.startswith(fn):
                        parts = [fn, namel[len(fn):]]
                    elif namel.endswith(fn):
                        parts = [fn, namel[:len(namel)-len(fn)]]
                if len(parts) == 1 and ln:
                    if namel.startswith(ln):
                        parts = [namel[len(ln):], ln]
                    elif namel.endswith(ln):
                        parts = [namel[:len(namel)-len(ln)], ln]

                if len(parts) == 2:
                    if fn and parts[0]==fn:
                        ln = parts[1]
                    elif fn and parts[1]==fn:
                        ln = parts[0]
                    elif ln and parts[1]==ln:
                        fn = parts[0]
                    elif ln and parts[0]==ln:
                        fn = parts[1]
                    else:
                        fn, ln = parts
                elif len(parts) > 2:
                    fn, ln = parts[0], parts[-1]
                    
            if not cc:
                cc = ''

            fo.write(f'{qid}\t{name}\t{fn}\t{ln}\t{gender}\t{desc}\t{cc}\t{dob}\t{image}\t{sort}\n')
fo.close()
    

In [ ]:
fo = open('/backup/wikidata/wiki_parsed.tsv', 'w')
fi = open('/backup/wikidata/wiki_person.tsv', 'r')
fi.readline()
fo.write(f"qid\tname\tprob\ttitle\tfn\tln\tfn_org\tln_org\n") 

for i, l in tqdm(enumerate(fi), total=9928121 ):
    qid, name, fn_org, ln_org, gender, desc, cc, dob, image, sort = l.strip('\n').split('\t')
    rec = parse(name)
    titles, fn, ln = [], [], []
    prob = -1 
    if rec['tags'] and rec['tags'][0]['prob']>0.2:
        prob = rec['tags'][0]['prob']
        for tok, typ in zip(rec['tags'][0]['tokens'], rec['tags'][0]['labels']):
            if typ[:5] == 'title':
                titles.append(tok)
            elif typ[:2] == 'fn':
                fn.append(tok)
            elif typ[:2] == 'ln':
                ln.append(tok)
    fo.write(f"{qid}\t{name}\t{prob}\t{','.join(titles)}\t{','.join(fn)}\t{','.join(ln)}\t{fn_org}\t{ln_org}\n") 
fo.close()

  0%|          | 0/9928121 [00:00<?, ?it/s]

In [ ]:
!wc -l /backup/wikidata/wiki_person.tsv

In [ ]:
uk = 0
fn = '/projekti/mondonomo/books/thai/thai_wiki.csv'
fo = open(fn, 'w')
fo2 = open(fn.replace('thai_', 'nonthai_'), 'w')
for i, l in enumerate(open('/backup/wikidata/wiki_person.jsonl')):
    j = json.loads(l)
    cc = get_wiki_cc(j['country'], j['birth_place'], j['language'])
    if 'en' in j['l'] and 'th' in j['l'] and cc:
        native = j["name_native"][0] if j["name_native"] else ''
        en = j["l"]["en"][0].replace(',',' ')
        th = j["l"]["th"][0].replace(',',' ')
        if cc == 'TH':
            fo.write(f'{j["id"]},{cc},{en},{th}\n')
        else:
            fo2.write(f'{j["id"]},{cc},{en},{th}\n')
        uk += 1
fo.close()
!head /projekti/mondonomo/books/thai/thai_wiki.csv

In [ ]:
from tqdm.notebook import tqdm
uk = 0
fn = '/backup/wikidata/hr_wiki.csv'
fo = open(fn, 'w')
fo2 = open(fn.replace('hr_', 'nonhr_'), 'w')
for l in tqdm(open('/backup/wikidata/wiki_person.jsonl'), total=10_044_571):
    j = json.loads(l)
    cc = get_wiki_cc(j['country'], j['birth_place'], j['language'])
    if 'en' in j['l'] and 'hr' in j['l'] and cc:
        native = j["name_native"][0] if j["name_native"] else ''
        en = j["l"]["en"][0].replace(',',' ')
        th = j["l"]["hr"][0].replace(',',' ')
        if cc == 'HR':
            fo.write(f'{j["id"]},{cc},{en},{th}\n')
        else:
            fo2.write(f'{j["id"]},{cc},{en},{th}\n')
        uk += 1
fo.close()
!head $fn

In [ ]:
parse('George Washington')